<a href="https://colab.research.google.com/github/SuperCoolCucumber/CompLing/blob/main/CompLing_exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pymorphy2
from pymystem3 import Mystem
import numpy as np
import razdel
import textdistance

import gensim
import tensorflow as tf
from transformers import TFAutoModel
from transformers import AutoTokenizer

In [20]:
import requests
r = requests.get(
    'https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/anna_karenina.txt'
)

# работайте с этими предложениями
sentences = r.text.split('\n')
sentences = [sent for sent in sentences if len(sent) > 10]

## Задание 1

In [21]:
mystem = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [22]:
text = "Зонды будут запущены в космос в 2029 году и отправятся ко второй точке Лагранжа в системе Солнце"

In [23]:
morph = pymorphy2.MorphAnalyzer()
words = [morph.parse(token) for token in text.split()]

In [25]:
tags = []

In [26]:
for i in range(len(words)):
    tags.append(words[i][0].tag.POS)

In [27]:
tags

['NOUN',
 'VERB',
 'PRTS',
 'PREP',
 'NOUN',
 'PREP',
 None,
 'NOUN',
 'CONJ',
 'VERB',
 'PREP',
 'ADJF',
 'NOUN',
 'NOUN',
 'PREP',
 'NOUN',
 'NOUN']

## Задание 2

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

In [29]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sentences)

In [30]:

feature_names = vectorizer.get_feature_names()
response = vectorizer.transform([sentences[0]])
feature_array = np.array(feature_names)
tfidf_sorting = np.argsort(response.toarray()).flatten()[::-1]

n = 3
top_n = feature_array[tfidf_sorting][:n]
top_n

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


array(['устои', 'насыщенное', 'приметами'], dtype='<U23')

## Задание 3

In [37]:
from razdel import tokenize 

In [38]:
a = []
for sentence in sentences:
    tokens = list(tokenize(sentence))
    a.append([_.text.lower() for _ in tokens])


def ngrammer(tokens, stops, n=2):
    ngrams = []
    tokens = [token for token in tokens if token not in stops]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

In [39]:
def collect_stats(corpus, stops):
    ## соберем статистики для отдельных слов
    ## и биграммов
    
    unigrams = Counter()
    bigrams = Counter()
    
    for sent in corpus:
        unigrams.update(sent)
        bigrams.update(ngrammer(sent, stops, 2))
    
    return unigrams, bigrams
def score_bigrams(unigrams, bigrams, scorer, threshold=-100000, min_count=1):
    ## посчитаем метрику для каждого нграмма
    bigram2score = Counter()
    len_vocab = len(unigrams)
    for bigram in bigrams:
        score = scorer(unigrams[bigram[0]], unigrams[bigram[1]], 
                       bigrams[bigram], len_vocab, min_count)
        
        ## если метрика выше порога, добавляем в словарик
        if score > threshold:
            bigram2score[bigram] = score
    
    return bigram2score

In [40]:
unigrams, bigrams = collect_stats(a, [])

In [41]:
def scorer_simple(word_count_a, word_count_b, bigram_count, *args):
    try:
        score = bigram_count/((word_count_a+word_count_b))
    
    except ZeroDivisionError:
        return 0
    
    return score

In [45]:
bigram2score = score_bigrams(unigrams, bigrams, scorer_simple)

In [43]:
for b in bigram2score:
    if b[0] == 'красный':
      print(b, bigram2score[b])

('красный', 'мешочек') 0.2777777777777778
('красный', 'огонь') 0.05263157894736842
('красный', ',') 5.082592121982211e-05
('красный', 'платок') 0.05555555555555555
('красный', '.') 6.033546518643659e-05


## Задание 4

In [44]:
words = words = ["решение","ршеение","ренешик","рещиние","ришение"]
for i in words:
  for j in words:
    dl = textdistance.damerau_levenshtein.distance(i, j)
    l = textdistance.levenshtein.distance(i, j)
    if dl!= l:
      print(i,j)

решение ршеение
ршеение решение


## Задание 5

In [3]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

model = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased")

In [8]:
tokenized = tokenizer.tokenize(sentences, is_split_into_words=True)

In [5]:
#tokenized = [tokenizer.tokenize(sent) for sent in sentences]

Token indices sequence length is longer than the specified maximum sequence length for this model (643 > 512). Running this sequence through the model will result in indexing errors


In [9]:
from gensim.models import FastText

In [7]:
model = FastText(sentences=tokenized, vector_size=300, window=5, min_n=2, max_n=7)

In [11]:
model.wv.most_similar('каренина')

[('##ка', 0.6053786873817444),
 ('а', 0.5131438374519348),
 ('##ович', 0.3983028829097748),
 ('##на', 0.394869327545166),
 (')', 0.31459611654281616),
 ('landau', 0.2750604748725891),
 ('–', 0.26895037293434143),
 ('##евич', 0.2547837495803833),
 ('cher', 0.24755364656448364),
 ('pardon', 0.2393816113471985)]